In [7]:
# Imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.stats import distributions as pdf
import matplotlib.pyplot as plt
import sklearn.datasets as dset

# Bayesian Linear Regression

Assuming, we have some data generating function $y$. However, the given training samples differ from the true function due to noise. Make another assumption, which is, that the noise is normal distributed, the following model is obtained: <br><br>
$\mathbf{y}  \sim \mathbf{X^T}\mathbf{\theta}+\mathbf{B}\mathcal{N}(\mathbf{0},\mathbf{I}) 
=\mathcal{N}(\mathbf{X^T}\mathbf{\theta},\mathbf{\Sigma})$ <br>

Next, the prior distribution over the parameters needs to be specified. 


Prior over weights: $p(w)=N(w|0,\alpha^{-1})$

Likelihood function:
$p(y|x,w,\beta) = N(y|w^Tx,\beta^{-1})$

In [9]:
def create_noisy_regression(num_data_samples):
    np.random.seed(42)
    x = np.random.uniform(-1,1,num_data_samples)
    ground_truth_weights = np.array((-0.3,0.5))
    targets =  ground_truth_weights[0] + ground_truth_weights[1]*x.reshape(-1,) 
    noisy_targets = targets + 0.2 *np.random.randn(x.size)
    return x, noisy_targets

In [10]:
def weight_posterior(x, y, alpha, beta):
    x = np.hstack((np.ones((x.size,1)),x[:,None]))
    posterior_w_precision = alpha+beta*x.T@x
    posterior_w_cov = np.linalg.inv(posterior_w_precision)
    posterior_w_mean = beta*posterior_w_cov@x.T@y[:,None]
    return posterior_w_mean, posterior_w_cov


In [20]:
def sample_weights(mean, cov, num_samples):
    w_sample = np.random.multivariate_normal(mean=mean.reshape(-1,), 
                                             cov=cov, size=num_samples)
    return w_sample

In [21]:
def plot_samples(w_sample):
    num_samples = w_sample.shape[0]
    x_plot = np.linspace(-1,1,100)
    for i in range(num_samples):
        plt.plot(x_plot,w_sample[i,0]+w_sample[i,1]*x_plot.reshape(-1,), c='royalblue')

In [26]:
num_datasample_widget = widgets.IntSlider(min=1, max=30, step=1, value=1)
num_weight_sample_widget = widgets.IntSlider(min=1, max=10, step=1, value=1)

def bayesian_framework(num_datasamples, num_weight_samples):
    x,y = create_noisy_regression(num_data_samples=num_datasamples)
    alpha = 2 #  weight precision
    beta = 25 #  target precision
    mean, cov = weight_posterior(x, y, alpha, beta)
    weight_samples = sample_weights(mean, cov, num_weight_samples)
    plot_samples(weight_samples)
    plt.scatter(x,y, c='k')
    plt.plot(np.linspace(-1,1,80),
             -0.3 + 0.5*np.linspace(-1,1,80), c='red')
    plt.xlim(-1,1)
    plt.xticks([])
    plt.yticks([])
    plt.ylim(-1,0.3)
    plt.show()

interact(bayesian_framework, 
         num_datasamples=num_datasample_widget,
         num_weight_samples=num_weight_sample_widget)
    

interactive(children=(IntSlider(value=1, description='num_datasamples', max=30, min=1), IntSlider(value=1, des…

<function __main__.bayesian_framework(num_datasamples, num_weight_samples)>